# 验证下在贪婪解码的情况下, transformers 和 vllm 的推理结果是否一致

In [1]:
# 基本设置
import torch

# model_dir = "/mnt/g/code/pretrain_model_dir/_modelscope/qwen/Qwen2-7B-Instruct"  # 加载需要 7分半
model_dir = "/root/home/my_model/Qwen2-7B-Instruct"  # 爷受伤了, wsl2 的 io 也太差了

torch_dtype_str = "bfloat16"
torch_dtype = torch.bfloat16
# torch_dtype = torch.float16

In [2]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer


device = "cuda" # the device to load the model onto
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch_dtype,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

print(model.dtype, model.device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


torch.bfloat16 cuda:0


In [17]:
def generate_greddy_chat(messages: list[str]):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        do_sample=False,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)

    return generated_ids, response


prompt = "你好"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
generate_greddy_chat(messages)

你好！很高兴能为你提供帮助。有什么问题或需要我解答的吗？


([tensor([108386,   6313, 112169,  26232, 106184,  99553, 100364,   1773, 104139,
           86119,  57191,  85106,  35946, 106185,   9370, 101037,  11319, 151645],
         device='cuda:0')],
 '你好！很高兴能为你提供帮助。有什么问题或需要我解答的吗？')

In [ ]:
prompts = ['vLLM is a high-throughput and memory-efficient inference and serving engine for LLMs.\n', 'Briefly describe the major milestones in the development of artificial intelligence from 1950 to 2020.\n', 'Compare and contrast artificial intelligence with human intelligence in terms of processing information.\n', 'Describe the basic components of a neural network and how it can be trained.\n', 'Write a short story about a robot that dreams for the first time.\n', 'Analyze the impact of the COVID-19 pandemic on global economic structures and future business models.\n', 'Explain the cultural significance of the Mona Lisa painting, and how its perception might vary in Western versus Eastern societies.\n', "Translate the following English sentence into Japanese, French, and Swahili: 'The early bird catches the worm.'\n"]

In [5]:
def generate_greddy_text(text: str):
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    print(model_inputs.input_ids,)
    generated_ids = model.generate(
        model_inputs.input_ids,
        do_sample=False,
        max_new_tokens=30
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)

    return generated_ids, response

# generate_greddy_text("你好,世界")

In [6]:
example = "vLLM is a high-throughput and memory-efficient inference and serving engine for LLMs.\n"
example = "你好,世界"
generate_greddy_text(example)

tensor([[108386,     11,  99489]], device='cuda:0')


/root/home/anaconda3/envs/vllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/home/anaconda3/envs/vllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/root/home/anaconda3/envs/vllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


!——记2017级新生开学典礼

金秋九月,丹桂飘香。在这收获的季节里,我们迎来了


([tensor([     0,   8545,  40814,     17,     15,     16,     22,  52334, 102900,
          107823, 113953,    271,  34230, 100057,  99609,   9754,     11, 100721,
          100877, 104712,  99662,   1773, 102332, 104619,   9370, 105419,  69249,
              11,  97639, 108646], device='cuda:0')],
 '!——记2017级新生开学典礼\n\n金秋九月,丹桂飘香。在这收获的季节里,我们迎来了')